<a href="https://colab.research.google.com/github/wolffg777/descriptors_to_music_rec/blob/main/MusicRec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
WORD2VEC KERAS CLASSIFICATION PROBLEM

- word2vec on descriptors and genres
  - first just working
  - then working out OOV (out of vocabulary) problems #fastText maybe ask chatgpt
- testing it to see improvement
- implementing dates (as continuous)
- easier things first
- spotify data integration


-brainstorm an idea for the name of the independent study on askbanner (limited to 30 characters)



In [ ]:
from google.colab import files
files.upload()
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d tobennao/rym-top-5000
!unzip rym-top-5000.zip
!pip install spotipy

KeyboardInterrupt: ignored

In [35]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

ImportError: ignored

In [ ]:
df = pd.read_csv('rym_clean1.csv')
# df.head(1)

# Remove rating_count, review_count, and avg_rating columns
df = df.drop(columns=['avg_rating'])
df = df.drop(columns=['rating_count'])
df = df.drop(columns=['review_count'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['position'])
df = df.drop(columns=['release_type'])
df.head(5)

,release_name,artist_name,release_date,primary_genres,secondary_genres,descriptors
0,OK Computer,Radiohead,1997-06-16,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ..."
1,Kid A,Radiohead,2000-10-03,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp..."
2,The Dark Side of the Moon,Pink Floyd,1973-03-23,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi..."
3,Loveless,My Bloody Valentine,1991-11-11,"Shoegaze, Noise Pop","Dream Pop, Neo-Psychedelia","noisy, ethereal, atmospheric, romantic, love, ..."
4,My Beautiful Dark Twisted Fantasy,Kanye West,2010-11-22,"Pop Rap, Hip Hop",Art Pop,"epic, boastful, passionate, sampling, hedonist..."


In [ ]:
#reformat descriptors properly
df['descriptors'] = df['descriptors'].str.split(', ')
desc_words = df['descriptors']

In [ ]:
# Word2Vec for descriptors
word2vec_model = Word2Vec(desc_words, min_count=5)
vocab = word2vec_model.wv.index_to_key
vocab_size = len(vocab)

In [ ]:
def descriptor_to_vec(descriptor):
    vectors = [word2vec_model.wv[word] for word in descriptor if word in vocab]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

#solution here: mean of vectors, would prefer a different approach
df['descriptor_avg_vec'] = df['descriptors'].apply(descriptor_to_vec)

# Padding descriptor vectors, basically making them all the same length
padded_descriptors = pad_sequences(df['descriptor_avg_vec'].tolist(), maxlen=10, dtype='float32')

In [ ]:
encoder = LabelEncoder()
df['release_name_label'] = encoder.fit_transform(df['release_name'])

# Prepare training data
X = np.array(padded_descriptors)
y = df['release_name_label'].values

In [ ]:
# print(X.shape)
# print(y.shape)

In [38]:
# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=padded_descriptors.shape[1]),
    LSTM(128),
    Dense(len(df['release_name'].unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=25, batch_size=64)

Epoch 1/25
79/79 [==============================] - 7s 58ms/step - loss: 8.4835 - accuracy: 0.0000e+00
Epoch 2/25
79/79 [==============================] - 3s 44ms/step - loss: 8.5326 - accuracy: 4.0000e-04
Epoch 3/25
79/79 [==============================] - 3s 43ms/step - loss: 8.5107 - accuracy: 2.0000e-04
Epoch 4/25
79/79 [==============================] - 3s 44ms/step - loss: 8.5123 - accuracy: 0.0000e+00
Epoch 5/25
79/79 [==============================] - 5s 58ms/step - loss: 8.5099 - accuracy: 2.0000e-04
Epoch 6/25
79/79 [==============================] - 3s 42ms/step - loss: 8.5090 - accuracy: 8.0000e-04
Epoch 7/25
79/79 [==============================] - 3s 42ms/step - loss: 8.5097 - accuracy: 6.0000e-04
Epoch 8/25
79/79 [==============================] - 4s 54ms/step - loss: 8.5084 - accuracy: 8.0000e-04
Epoch 9/25
79/79 [==============================] - 4s 44ms/step - loss: 8.5083 - accuracy: 8.0000e-04
Epoch 10/25
79/79 [==============================] - 3s 42ms/step - loss:

things to do: stricter filtering of descriptors
one-hot the genres, word2vec (number of dimensions depends on embedding model, basic word2vec is 30 - 100), the descriptors and average the value and put it in space. tutorials on youtube seem good.

try to find a taxonomy of genres online, if not split up words and vaguely separating the musical words

Partial credit accuracy/reward system where "radiohead" for atmospheric even if the album recommended is not atmospheric

In [ ]:
import os
import base64
from requests import post, get
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials


os.environ['SPOTIPY_CLIENT_ID'] = '08a0640020ff4e5d9713aaeacfbd1886'
os.environ['SPOTIPY_CLIENT_SECRET'] = '81cefe24f94b4f24af9fe1ad2e70ef4a'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://example.com/callback'

#Authorization Code Flow
scopes = "user-library-read user-top-read user-follow-read"

sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          scope=scopes, open_browser=False))

In [ ]:
#Get Specified User's Playlists
playlists = sp.user_playlists('wolffg7')

while playlists:
    for i, playlist in enumerate(playlists['items']):
        #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))

    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None


In [ ]:
album_name = "OK Computer"
artist_name = "Radiohead"

# Search for the album
results = sp.search(q=f"album:{album_name} artist:{artist_name}", type="album")

album_id = results['albums']['items'][0]['id']

album_tracks = sp.album_tracks(album_id)

for track in album_tracks['items']:
    print(track['name'])


model architecture: pytorch simple neural network (feedforward neural network).

get a model working before break, even if its simpler (in terms of model architecture and pre-processing).

in recommendation stage, top n choices of different artists because otherwise it will just recommend songs from the same album
|
also figure out a way to vectorize/umbrella categories for genres
|
add column for frequency of each genre like
col 1 : space rock
col 2 : rock
col 3 : frequency 3
|
also separate the words in the descriptors (ie "malevocals" should be "male vocals")

still do this, but only useful for bringing in new information that is song specific, like lyrics and popularity

In [ ]:
# CODE GRAVEYARD

# # Flatten the lists in the "descriptors" column
# flattened_descriptors = df['descriptors'].explode()

# # Count the number of unique strings
# unique_count = flattened_descriptors.nunique()

# flattened_descriptors.head(5)

# # Use value_counts to count the frequency of elements in the column
# element_counts = flattened_descriptors.value_counts()

# # Print the result
# print(element_counts)



# # One-hot-encoding for DESCRIPTORS

# # Turn descriptors into a list of strings
# df['descriptors'] = df['descriptors'].str.split(',')

# descriptors_encoded = df['descriptors'].str.join('|').str.get_dummies()

# descriptor_frequencies = descriptors_encoded.iloc[:, 2:].sum()

# # print (descriptor_frequencies)

# rarity_threshold = 10

# # Clean column names by stripping spaces
# descriptors_encoded.columns = descriptors_encoded.columns.str.strip()

# rare_descriptors = descriptor_frequencies[descriptor_frequencies < rarity_threshold]

# # print(rare_descriptors)
# # print(len(rare_descriptors))